## Demographics

a. Identify IDs with 3T and 7T  
b. Extract clinical information for epilepsy patients  
c. Extract demographic information for all participants  


Note participant grouping:
- CTRL : no known epilepsy (identified by ID code: 3T: HC; 7T: PNC, Pilot)

- (m)TLE : temporal lobe epilepsy
    - TLE_L : TLE left lateralized (including L>R)
    - TLE_R : TLE right lateralized (including R>L)
    - TLE_U : TLE with unknown lateralization
    - mTLE_L : mTLE left lateralized
    - mTLE_R : mTLE right lateralized
- FLE : focal epilepsy that is not in temporal lobe
    - FLE_L
    - FLE_R
- UKN : Focal epilepsy with focus in unknown lobe
    - UKN_L : Unknown lobe with known L lateralization
    - UKN_R : Unknown lobe with known R lateralization
    - UKN_U : Unknown lobe with unknown lateralization
- MFCL : multifocal epilepsy
    - MFCL : multifocal epilepsy


In [11]:
import pandas as pd
import os
import sys
import importlib
import re
import numpy as np

sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code")
#sys.path.append("/host/verges/tank/data/daniel/")
from Utils import id, gen, t1

# I. Functions

In [ ]:
def group(df, ID_col="MICS_ID", out_col="grp", save_pth=None, MFCL_col="Epilepsy classification:Focal,Generalized", lobe_col="Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)", lat_col="Lateralization of epileptogenic focus"):
    """
    Requires pandas as pd

    Inputs:
    df: str or pd.DataFrame
        demographics data
    ID_col: str
        Column name for the ID
    out_col: str
        Column name for the output group classification. 
        Options: 'grp' returns high-level grouping. All other col_names return detailed grouping.
    MFCL_col: str
        Column name for the multifocal classification
    lobe_col: str  
        Column name for the lobe classification
    lat_col: str
        Column name for the lateralization classification

    Outputs:
    df: pd.DataFrame
        DataFrame with the group classification added
    """

    import pandas as pd
    import os


    print("[group] Identifying participant groups")
    # check if df is a string (path) or dataframe
    if isinstance(df, str):
        # check if file exists
        if not os.path.isfile(df):
            raise ValueError(f"[group] Error: {df} does not exist.")
        
        # read in file
        df = pd.read_csv(df, dtype=str)
    elif isinstance(df, pd.DataFrame):
        pass
    else:
        raise ValueError("[group] Error: df must be a string (path) or dataframe")

    df[out_col] = df.apply(
        lambda row: f"PATTERN NOT RECOGNIZED: lobe={row.get(lobe_col, None)}, lat={row.get(lat_col, None)}, MFCL={row.get(MFCL_col, None)}",
        axis=1
    )

    if ID_col == "MICS_ID":
        ctrl_ptrn = ["HC"]
    elif ID_col == "PNI_ID":
        ctrl_ptrn = ["Pilot", "PNC"]
    else:
        raise ValueError("[group] Error: ID_col must be 'MICS_ID' or 'PNI_ID'")

    
    if out_col == "grp":
        print("\tReturning highlevel grouping to column: ", out_col)
       
        df.loc[df[ID_col].astype(str).str.contains('|'.join(ctrl_ptrn), na=False), out_col] = 'CTRL'
        
        # TLE and mTLE: L, left, R, right, unclear
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & 
            (df[lat_col].astype(str).str.lower().str.contains('l|left|r|right|l>r|r>l|bl|bilateral|unclear', na=False)), 
            out_col
        ] = 'TLE'
        
        # FLE: L, R, right, unclear
        df.loc[
            (df[lobe_col] == 'FLE') & 
            (df[lat_col].astype(str).str.contains('l|r|right|unclear', case=False, na=False)), 
            out_col
        ] = 'FLE'
        
        # PLE: L, R, right, unclear
        df.loc[
            ((df[lobe_col] == 'PLE')) &
            (df[lat_col].astype(str).str.contains('l|r|right|unclear', case=False, na=False)),
            out_col
        ] = 'PLE'

        # PQLE: L, R, right, unclear
        df.loc[
            ((df[lobe_col] == 'PQLE')) &
            (df[lat_col].astype(str).str.contains('l|r|right|unclear', case=False, na=False)),
            out_col
        ] = 'PQLE'

        # Unclear: L, R, right, unclear
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col] == 'L'), out_col] = 'UKN'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')), out_col] = 'UKN'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col].astype(str).str.contains('unclear', na=False)), out_col] = 'UKN'
        
        # Multifocal
        df.loc[(df[MFCL_col] == 'Multifocal'), out_col] = 'MFCL'
    
    else:  
        print("\tReturning detailed grouping to column: ", out_col) 
        
        df.loc[df[ID_col].astype(str).str.contains('|'.join(ctrl_ptrn), na=False), out_col] = 'CTRL'
        
        # Combine TLE and mTLE, label as TLE
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & (df[lat_col] == 'L'),
            out_col
        ] = 'TLE_L'
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')),
            out_col
        ] = 'TLE_R'
        df.loc[
            (df[lobe_col].astype(str).str.lower().isin(['tle', 'mtle'])) & (df[lat_col].astype(str).str.contains('unclear', na=False)),
            out_col
        ] = 'TLE_U'

        df.loc[(df[lobe_col] == 'FLE') & (df[lat_col] == 'L'), out_col] = 'FLE_L'
        df.loc[(df[lobe_col] == 'FLE') & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')), out_col] = 'FLE_R'

        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col] == 'L'), out_col] = 'UKN_L'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & ((df[lat_col] == 'R') | (df[lat_col].astype(str).str.lower() == 'right')), out_col] = 'UKN_R'
        df.loc[(df[lobe_col].astype(str).str.contains('unclear', na=False)) & (df[lat_col].astype(str).str.contains('unclear', na=False)), out_col] = 'UKN_U'
        
        df.loc[(df[MFCL_col] == 'Multifocal'), out_col] = 'MFCL'
        df.loc[(df[lobe_col] == 'TLE') & (df[lat_col] == 'L>R'), out_col] = 'TLE_L'
        df.loc[(df[lobe_col] == 'TLE') & (df[lat_col] == 'R>L'), out_col] = 'TLE_R'
        df.loc[(df[lobe_col] == 'TLE') & (df[lat_col] == 'BL'), out_col] = 'TLE_BL'
    
    return df

def mergeCols(df):
    """
    Merge columns with similar names (case-insensitive, ignoring whitespace) by prioritizing non-null values.
    For each group of similar columns, select the column with the shortest name (or lowercase if tied),
    and fill its missing values with values from the other columns in the group (row-wise).
    Drops the other columns in the group.

    Input:
        df: DataFrame with potential duplicate/similar columns

    Output:
        df: DataFrame with merged columns
    """

    # Normalize column names: lower case, strip whitespace
    def norm(col):
        return re.sub(r'\s+', '', col).lower()

    cols = list(df.columns)
    norm_map = {}
    for col in cols:
        key = norm(col)
        norm_map.setdefault(key, []).append(col)

    for group in norm_map.values():
        if len(group) > 1:
            # Choose the column with shortest name, or lowercase if tied
            group_sorted = sorted(group, key=lambda x: (len(x), x.lower()))
            main_col = group_sorted[0]
            other_cols = [c for c in group if c != main_col]
            print(f"[mergeCols] Merging columns: {group} -> '{main_col}'")
            # Fill missing values in main_col from other columns, row-wise
            df[main_col] = df[group].bfill(axis=1).iloc[:, 0]
            # Drop the other columns
            df.drop(columns=other_cols, inplace=True)
    return df

def stdizeNA(df, missing_patterns=None, verbose=True):
    """
    Replace various missing value patterns in the DataFrame with np.nan.

    Parameters:
        df: pd.DataFrame
        missing_patterns: list or None
            List of patterns to treat as missing. If None, uses a default set.
        verbose: bool
            If True, prints detailed information about changes made.

    Returns:
        df: pd.DataFrame with standardized missing values
    """

    # Default patterns for missing values
    default_patterns = [
        '', ' ', 'nan', 'NaN', 'NAN', 'null', 'NULL', '?', 'NA', 'na', 'n/a', 'N/A', '.', '-', '--', 'missing', 'MISSING'
    ]
    
    if missing_patterns is not None:
        patterns = set(default_patterns) | set(missing_patterns)
    else:
        patterns = set(default_patterns)

    if verbose:
        # Print every value that WOULD be changed (i.e., would be replaced with np.nan)
        mask = df.isin(patterns)
        changed = mask.stack()[lambda x: x].index.tolist()
        if changed:
            print(f"[stdizeNA] The following values will be changed given match to any of: {patterns}")
            for idx, col in changed:
                old_val = df.at[idx, col]
                print(f"\t[MICSId={df.at[idx, 'MICS_ID'] if 'MICS_ID' in df.columns else 'NA'}={df.at[idx, 'PNI_ID'] if 'PNI_ID' in df.columns else 'NA'} study: {df.at[idx, 'study'] if 'study' in df.columns else 'NA'}-ses{df.at[idx, 'SES'] if 'SES' in df.columns else 'NA'}] {{{col[:10]}}}: {old_val} --> NaN")

    # Replace all matching patterns with np.nan
    df = df.replace(list(patterns), np.nan)
    num_changed = df.isna().sum().sum()
    print(f"[stdizeNA] Standardized {num_changed} missing values")
    return df


def carryVals(df, var):
    """
    Carry forward and backward fill demographic variables for each participant.

    For each participant (grouped by 'MICS_ID'), if multiple unique non-null values are found for the variable:
      - Use the value with the shortest number of characters.
      - If there is a tie, use the first value encountered.
      - Print a warning indicating all found values and which one was used, including study and session number if available.

    Input:
        df: DataFrame containing demographic information
        var: Column name to carry forward/backward fill

    Output:
        df: DataFrame with the variable filled per participant
    """
    df[var] = df.groupby('MICS_ID')[var].transform(lambda x: x.ffill().bfill())
    # Check for multiple unique values for the same ID
    dupes = df.groupby('MICS_ID')[var].nunique()
    multi_val_ids = dupes[dupes > 1].index
    for mid in multi_val_ids:
        vals = df[df['MICS_ID'] == mid][var].dropna().unique()
        # Choose value with shortest number of characters, if tie, use first
        vals_sorted = sorted(vals, key=lambda v: (len(str(v)), str(v)))
        chosen = vals_sorted[0]
        pni_id = df[df['MICS_ID'] == mid]['PNI_ID'].iloc[0] if 'PNI_ID' in df.columns else ''
        study = df[df['MICS_ID'] == mid]['study'].iloc[0] if 'study' in df.columns else ''
        ses = df[df['MICS_ID'] == mid]['SES'].iloc[0] if 'SES' in df.columns else ''
        used_idx = list(vals).index(chosen) + 1
        print(f"\t[carryVals] WARNING: [{mid}={pni_id} Study: {study}-ses{ses}] {var} : {' | '.join(map(str, vals))} --> {chosen}")
    
    # Use the chosen value for each group
    def choose_val(x):
        non_nulls = x.dropna()
        if len(non_nulls.unique()) > 1:
            vals_sorted = sorted(non_nulls.unique(), key=lambda v: (len(str(v)), str(v)))
            return vals_sorted[0]
        elif len(non_nulls) > 0:
            return non_nulls.iloc[0]
        else:
            return x
    df[var] = df.groupby('MICS_ID')[var].transform(choose_val)
    return df


In [13]:
def main(sheets, save_pth=None, save_name="demo"):
    """
    Run all functions to generate demographic data for 3T-7T participants.

    input:
        sheets: list of dictionarys with source sheet information (path to sheet, key columns to extract
        save_pth: path to save the output demographic file

    outputs:
        list of IDs with paired 3T-7T data
        sheet with each row as separate session and with associated demographic information
    """

    import os
    sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code") # on mac
    #sys.path.append("/host/verges/tank/data/daniel/") # on lab computer
    from Utils import id, t1

    import importlib
    importlib.reload(id)
    importlib.reload(t1)
    
    out = id.ids3T7T(sheets, save_pth=None) # determine participants with 3T and 7T data
    id_cols = out.columns

    out = id.id_visits(sheets, out, save_pth=None) # get all sessions for these participants
    print("[main] There are ", out['MICS_ID'].nunique(), " unique participants with a total of ", out.shape[0], " sessions in input datasheet.")

    out = t1.demo(sheets, out, save_pth=None) # add demographic info

    # remove rows with missing date. If only data from one study remains after this removal, then remove participant completely.
    n_missing_dates = out['Date'].isna().sum()
    if n_missing_dates != 0:
        missing_dates = out[out['Date'].isna()]
        count_7T = (missing_dates['study'] == '7T').sum()
        count_3T = (missing_dates['study'] == '3T').sum()
        print(f"\n[main] WARNING. {n_missing_dates} rows have missing scan sdates:\n\t{count_7T} from 7T\n\t{count_3T} from 3T")
        print("[main] Removing the following participants (study of the removed row):")
        missing_ids = missing_dates['MICS_ID'].unique()
        out = out.dropna(subset=['Date'])
        for mid in missing_ids:
            # there should be at least one row with each study code
            studies = out[out['MICS_ID'] == mid]['study'].unique() # remaining studies for that ID
            if len(studies) < len(out['study'].unique()): # if this has ID has fewer unique studies than those present in the data, remove all rows for that ID
                print(f"\t{mid} (removed study: {studies})")
                out = out[out['MICS_ID'] != mid]
        print("[main] After cleaning for missing scan dates, there are ", out['MICS_ID'].nunique(), " unique participants making with a total of ", out.shape[0], " sessions in input datasheet.")
    
    # save out
    if save_pth is not None:
        import datetime

        save_name_tmp = f"{save_pth}/ids3T7T_{datetime.datetime.now().strftime('%d%b%Y')}.csv"
        toSave = out[id_cols].drop_duplicates()
        toSave.to_csv(save_name_tmp, index=False)
        print("[main] Saved list of paired ids to: ", save_name_tmp)

    dob_col = None
    for sheet in sheets: # find DOB from sheet dictionary. If multiple sheets have DOB, use the first one found
        if 'DOB' in sheet and sheet['DOB'] is not None:
            dob_col = sheet['DOB']
            #print(f"[main] Found DOB column in sheet {sheet['NAME']}: {dob_col}")
            break

    if dob_col is None:
        print("[main] WARNING. No DOB column found in any input sheet. Age will not be computed.")
    
    out = stdizeNA(out)
    out = mergeCols(out)
    print("[main] Merged columns with the same name. Current columns (sorted): \n\t", sorted(out.columns.tolist(), key=lambda x: x.lower()))

    # Fill missing demo variables for each participant, warn if multiple unique values exist
    demo_vars = ['dob', 'sex', 'gender', 'handedness', 'ethnicity'] # these demo vars should be key values in the sheet dictionaries
    print("[main] Filling missing demographic values for variables: ", demo_vars)

    for var in demo_vars:
        out = carryVals(out, var)

    out = t1.dateDif(out, [dob_col, "Date"], "age", save=False) # compute age

    out = group(out, out_col="grp", ID_col="MICS_ID", save_pth=None) # assign high level groups
    out = group(out, out_col="grp_detailed", ID_col="MICS_ID", save_pth=None) # assign detailed groups

    if save_pth is not None:
        import datetime

        save_name = f"{save_pth}/{save_name}_{datetime.datetime.now().strftime('%d%b%Y')}.csv"
        out.to_csv(save_name, index=False)
        print("[main] Saved to: ", save_name)
    else:
        print("[main] WARNING. Not saving demographics sheet to file. To save, please provide a path to save_pth")
        
    return out

## II. Make participant demographics sheet

In [14]:
src_dir = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/PT/sources" # path to directory with source pt sheets

# For each sheet, must define NAME, PATH, SHEET, ID_7T, ID_3T. 
# All other keys are those to be extracted.
# The same variables should have the same key names across sheets.

PNI = {
    'NAME': 'PNI',
    'PATH': f'{src_dir}/MICA_PNI_27Aug2025.xlsx', # 7T controls
    'SHEET': 'all', # name of sheet in file
    'ID_7T': 'ID_PNI', 
    'ID_3T': 'ID_MICs',
    'Ses_7T': 'session',
    'Date_7T': 'scanDate',
    'study': '7T',
    'DOB': 'dob',
    'Sex': 'sex',
    'Gender': 'gender',
    'Hand': 'handedness',
    'Eth': 'ethnicity',
    'Language': 'language',
    'Job': 'employment',
    'Edu': 'education',
    'LastSz': 'lastSeizure'
}

MICs = {
    'NAME': 'MICs',
    'PATH': f'{src_dir}/MICA-MTL-3T_27Aug2025.xlsx', # 3T controls
    'SHEET': 'Sheet1', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'Study_name',
    'Ses_3T': 'Visit',
    'Date_3T': 'Scan_Date (D.M.Y)',
    'study': '3T',
    'Hand': 'Handed', 
    'Sex': 'AssignedSex',
    'Gender': 'GenderIdentity',
    'Height': 'HeightApprox',
    'Weight': 'WeightApprox',
    'Eth': 'Ethnicity',
    'Job': 'Employ',
    'Edu': 'YoE',
    'LastSz': 'Last seizure'
}

Clin = {
    'NAME': 'Clin',
    'PATH': f'{src_dir}/Clinical_27Aug2025.xlsx',
    'SHEET': 'clinical-database-detailed', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'participant_id',
    'Date_3T': None,
    'Gender': 'Gender',
    'Hand': 'Handedness',
    'Language': 'Language',
    'Job': 'Employment',
    'Edu': 'Education',
    'EpilepsyDxILAE': 'Epilepsy diagnosis based on ILAE',
    'EpilepsyClass': 'Epilepsy classification:Focal,Generalized',
    'FocusLat': 'Lateralization of epileptogenic focus',
    'FocusConfirmed': 'Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)',
    'EMUDischargeDx': 'Dx at EMU discharge ',
    'EMUAdmissionDate': 'EMU admission date(dd-mm-yy)',
    'AdmissionDuration': 'Duration of admission',
    'EpilepsyRiskFactors': 'Risk factors for epilepsy',
    'SeizureOnsetYr': 'Seizure onset (yr)',
    'DrugResistant': 'Drug resistant epilepsy at time of EMU admission',
    'NumASMsPrior': '# of ASMs prior current EMU admission',
    'PrevASMs': 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission',
    'NumASMOnAdmission': '# of ASM on admission',
    'ASMsOnAdmission': 'ASMs  on admission (name, doses (mg per day)',
    'GeneticTest': 'Genetic test (year,results)',
    'FDGPET': 'FDG.PET',
    'BaselineMRI': 'Baseline MRI (year,results)',
    'InvasiveExplorations': 'Invasive explorations (Y/N)',
    'NumSurgicalResections': '# of surgical resection/thermocoagulatin',
    'SurgicalResectionDateSite': 'Surgical resection date and site',
    'Histopathology': 'Histopatholgy',
    'Engel6mo': 'Engel classification (seizure outcomes at the 6 month )',
    'Engel1yr': 'Engel classification (seizure outcomes after 1 year from surgical resection)',
    'ILAEOutcome1yr': 'ILAE outcome after surgical resection by 1 yr',
    'NeuromodDevices': 'Neuromodulation devices'
    }


sheets = [PNI, MICs, Clin]
correspSheets = [PNI]

In [15]:
save_pth = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/outputs"
demo = main(sheets, save_pth=save_pth)

[ID_3T7T] Extracting IDs for participants with both 3T and 7T IDs
	Loading: PNI
	Skipping: MICs (missing ID_7T or ID_3T key)
	Skipping: Clin (missing ID_7T or ID_3T key)
[id_visits] Extracting visits for IDs in list
	Skipping: Clin (missing Ses_7T and Ses_3T key)


	PNI
		Extracting cols: ['ID_MICs', 'ID_PNI', 'scanDate', 'session']
	MICs
		Extracting cols: ['Study_name', 'Scan_Date (D.M.Y)', 'Visit']
		Finding corresponding ID_7T
[main] There are  63  unique participants with a total of  203  sessions in input datasheet.
[demo] Retrieving demographics data.
	Overlapping: ['Sex', 'Gender', 'Hand', 'Eth', 'Language', 'Job', 'Edu', 'LastSz', 'Date_3T']
	PNI
		merge_keys: ['ID_7T', 'ID_3T', 'Ses_7T']
	MICs
		merge_keys: ['ID_3T', 'Ses_3T']
	Clin
		merge_keys: ['ID_3T']
	Removing empty columns: ['YoE', 'WeightApprox', 'AssignedSex', 'Handed', 'Employ', 'Scan_Date (D.M.Y)', 'GenderIdentity', 'HeightApprox', 'Ethnicity'] [main] Saved list of paired ids to:  /Users/danielmendelson/Library/Clo

/var/folders/jz/knj2s5ld08xb9qtkljpdcl9r0000gn/T/ipykernel_78804/708700703.py:225: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[var] = df.groupby('MICS_ID')[var].transform(lambda x: x.ffill().bfill())
/var/folders/jz/knj2s5ld08xb9qtkljpdcl9r0000gn/T/ipykernel_78804/708700703.py:225: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[var] = df.groupby('MICS_ID')[var].transform(lambda x: x.ffill().bfill())
/var/folders/jz/knj2s5ld08xb9qtkljpdcl9r0000gn/T/ipykernel_78804/708700703.py:225: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, 

!! ENSURE THAT GROUPING OCCURED PROPERLY !!

In [16]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

demo

,MICS_ID,PNI_ID,study,SES,Date,lastSeizure,handedness,dob,language,employment,gender,sex,education,ethnicity,scanDate,"Genetic test (year,results)","ASMs on admission (name, doses (mg per day)",Dx at EMU discharge,ILAE outcome after surgical resection by 1 yr,"Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)",Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission,Seizure onset (yr),# of ASM on admission,Neuromodulation devices,Epilepsy diagnosis based on ILAE,Histopatholgy,Duration of admission,# of ASMs prior current EMU admission,Engel classification (seizure outcomes at the 6 month ),Drug resistant epilepsy at time of EMU admission,Surgical resection date and site,"Baseline MRI (year,results)",Lateralization of epileptogenic focus,EMU admission date(dd-mm-yy),Engel classification (seizure outcomes after 1 year from surgical resection),Invasive explorations (Y/N),Risk factors for epilepsy,FDG.PET,"Epilepsy classification:Focal,Generalized",age,grp,grp_detailed
0,HC129,Pilot013,7T,05,18.04.2024,NaN,L,NaN,en,Full time student,F,F,Master Studnet,Canadian,18.04.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
1,HC082,PNC003,7T,01,06.05.2022,NaN,R,17.09.1997,English,Full time student,F,F,PhD Student,Japanese,06.05.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.632444,CTRL,CTRL
2,HC082,PNC003,7T,03,13.03.2023,NaN,R,17.09.1997,English,Full time student,F,F,PhD Student,Japanese,13.03.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.483915,CTRL,CTRL
3,HC082,PNC003,7T,02,13.06.2022,NaN,R,17.09.1997,English,Full time student,F,F,PhD Student,Japanese,13.06.2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.736482,CTRL,CTRL
4,HC082,PNC003,7T,04,24.10.2023,NaN,R,17.09.1997,English,Full time student,F,F,PhD Student,Japanese,24.10.2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.099932,CTRL,CTRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,PX235,PNE033,3T,01,29.07.2025,NaN,R,10.01.1984,en,employee,F,F,high school diploma,Canadian,NaN,NaN,NaN,NaN,NaN,FLE,NaN,NaN,NaN,NaN,definite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,Focal,41.549624,FLE,FLE_R
208,PX236,PNE034,3T,01,31.07.2025,NaN,R,12.07.2007,fr,Etudiante au CEGEP (sciences de la nature),F,F,Secondaire,Cameroonian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Focal,18.053388,"PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=Focal","PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=Focal"
209,PX240,PNE037,3T,01,12.08.2025,NaN,R,11.11.2001,fr,Hospital Cleaner,F,F,Almost finished CEGEP,Algerian,NaN,Followed at the CHUM,Followed at the CHUM,periventricular nodular heterotopia-R fronto/fronto-temporal epilepsy,Followed at the CHUM,unclear,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,definite,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,unclear,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Focal,23.750856,UKN,UKN_U
210,PX242,PNE039,3T,01,20.08.2025,NaN,R,07.09.1947,NaN,NaN,NaN,M,NaN,Canadian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
demo[["MICS_ID", "PNI_ID", "SES", "grp", "grp_detailed"]]

In [ ]:
# print all cases with missing group
missing = demo[(demo["grp"] == '') | (demo["grp_detailed"] == '')]
if len(missing) > 0:
    missing[
    ["MICS_ID", "PNI_ID", "SES", "grp", "grp_detailed", 
        "Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)", 
        "Lateralization of epileptogenic focus"]
        ]
else:
    print("All assigned a group")

All assigned a group



# Get summary statistics by category

In [44]:
# print number per group, median age, number of 3T and 7T sessions in each group
#demo['grp'].value_counts()
# count unique participants per group
demo.groupby('grp')['MICS_ID'].nunique().sort_values(ascending=False)

grp
TLE                                                      19
CTRL                                                     16
UKN                                                       7
FLE                                                       6
PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=Focal     2
MFCL                                                      1
PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=nan       1
PLE                                                       1
PQLE                                                      1
Name: MICS_ID, dtype: int64

In [43]:
demo.groupby('grp_detailed')['MICS_ID'].nunique().sort_values(ascending=False)

grp_detailed
CTRL                                                                                       16
TLE_R                                                                                       8
TLE_L                                                                                       6
UKN_U                                                                                       4
FLE_L                                                                                       3
FLE_R                                                                                       3
TLE_BL                                                                                      3
PATTERN NOT RECOGNIZED: lobe=nan, lat=nan, MFCL=Focal                                       2
TLE_U                                                                                       2
UKN_L                                                                                       2
MFCL                                           

In [45]:
# print all grp=TLE with laterlaization
demo[demo['grp'] == 'TLE'][['MICS_ID', 'PNI_ID', 'SES', 'grp', 'grp_detailed', 'Lateralization of epileptogenic focus']].drop_duplicates().sort_values(['grp', 'grp_detailed', 'MICS_ID'])

,MICS_ID,PNI_ID,SES,grp,grp_detailed,Lateralization of epileptogenic focus
63,PX153,PNE009,a1,TLE,TLE_BL,BL
64,PX153,PNE009,a2,TLE,TLE_BL,BL
141,PX153,PNE009,01,TLE,TLE_BL,BL
57,PX176,PNE004,a1,TLE,TLE_BL,BL
151,PX176,PNE004,01,TLE,TLE_BL,BL
72,PX200,PNE015,a1,TLE,TLE_BL,BL
94,PX200,PNE015,a2,TLE,TLE_BL,BL
160,PX200,PNE015,01,TLE,TLE_BL,BL
65,PX168,PNE010,a1,TLE,TLE_L,L
146,PX168,PNE010,01,TLE,TLE_L,L


In [21]:
demo[['grp', 'study', 'SES']].drop_duplicates().sort_values(['grp', 'study', 'SES'])

,grp,study,SES
178,,3T,01
90,,7T,a1
101,CTRL,3T,01
104,CTRL,3T,02
115,CTRL,3T,03
170,CTRL,3T,04
1,CTRL,7T,01
3,CTRL,7T,02
2,CTRL,7T,03
4,CTRL,7T,04


In [12]:
# get median age per study in each group
demo.groupby(['grp', 'study'])['age'].median().sort_index(level='grp')

grp   study
CTRL  3T       28.284736
      7T       28.861054
FLE   3T       33.519507
      7T       28.969199
MFCL  3T       34.135524
      7T       34.214921
TLE   3T       30.970568
      7T       30.781656
UKN   3T       31.735797
      7T       31.786448
Name: age, dtype: float64

In [13]:
# show histograms for each group

def plot_histogram(data, group_col, age_col, title):
    """
    Overlay histograms of age distribution for each group in the DataFrame.

    Parameters:
    data (pd.DataFrame): DataFrame containing the data.
    group_col (str): Column name for the group classification.
    age_col (str): Column name for the age values.
    title (str): Title for the plot.
    """
    import matplotlib.pyplot as plt
    import pandas as pd

    plt.figure(figsize=(10, 6))

    # Convert age column to numeric, coerce errors to NaN
    ages = pd.to_numeric(data[age_col], errors='coerce')

    # Overlay histograms for each group
    groups = data[group_col].unique()
    for grp in groups:
        subset = data[data[group_col] == grp]
        age_vals = pd.to_numeric(subset[age_col], errors='coerce').dropna()
        plt.hist(age_vals, bins=10, alpha=0.5, label=grp)

    plt.xlabel('Age')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_histogram(demo, 'grp_highLvl', 'age', 'Age Distribution by Group')


KeyError: 'grp_highLvl'

<Figure size 1000x600 with 0 Axes>

# Create table 1

In [ ]:
import pandas as pd

df_pth = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/demo_23May2025.csv"
df = pd.read_csv(df_pth, dtype=str)
df.columns

In [ ]:
columns_to_keep = ['MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'handedness', 'education', 'gender', 'ethnicity', 'employment', 'sex', 'age', 'grp', 'Lateralization of epileptogenic focus', 'Dx at EMU discharge ', 'grp_detailed']
df_short = df[columns_to_keep]
print(df_short.shape)
df_short['age'] = pd.to_numeric(df_short['age'], errors='coerce')
df_short.head()

In [ ]:
# print rows for HC154
df_short[df_short['MICS_ID'] == 'HC154']
# remove rows with MICS_ID == 'HC154'
df_short = df_short[df_short['MICS_ID'] != 'HC154']

In [ ]:
import numpy as np
import pandas as pd

# for repeated rows, values in columns other than Date, age and study should be the same. If NaN then replace with the first non-NaN value in the group
df_short = (
    df_short.groupby('MICS_ID', group_keys=False)
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
)
df_short = df_short.infer_objects(copy=False)

# keep most recent session only (via Date column)
df_3T = df_short[df_short['study'] == '3T']
df_3T = df_3T.sort_values(by='Date').drop_duplicates(subset='MICS_ID', keep='last')
df_3T = df_3T.reset_index(drop=True)
df_3T = df_3T.sort_values(by='MICS_ID')


df_7T = df_short[df_short['study'] == '7T']
df_7T = df_7T.sort_values(by='Date').drop_duplicates(subset='MICS_ID', keep='last')
df_7T = df_7T.reset_index(drop=True)
df_7T = df_7T.sort_values(by='MICS_ID')

print(f"3T shape: {df_3T.shape}")
print(f"7T shape: {df_7T.shape}")
print(f"Unique MICS_IDs in 3T: {df_3T['MICS_ID'].nunique()} (IDs: {df_3T['MICS_ID'].unique()})")
print(f"Unique MICS_IDs in 7T: {df_7T['MICS_ID'].nunique()} (IDs: {df_7T['MICS_ID'].unique()})")

In [ ]:
df_3T

In [ ]:
df_7T

In [ ]:
df_clean = pd.concat([df_3T, df_7T], ignore_index=True)
df_clean


In [ ]:
# compute date difference between 3T and 7T scans by individual
df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')
df_clean['Time between scans (mths)'] = np.nan
# for each unique MICS_ID, compute the difference between the 3T and 7T dates. Return this value to both rows
for mics_id in df_clean['MICS_ID'].unique():
    dates = df_clean[df_clean['MICS_ID'] == mics_id]['Date']
    
    date_diff_months = abs((dates.max() - dates.min()).days / 30.44)  # average days per month, absolute value
    df_clean.loc[df_clean['MICS_ID'] == mics_id, 'Time between scans (mths)'] = date_diff_months

df_clean.sort_values(by='MICS_ID')

In [ ]:
# make histogram of time between scans
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['Time between scans (mths)'].dropna(), bins=30, kde=True)
plt.title('Time Between 3T and 7T Scans')

mean_val = df_clean['Time between scans (mths)'].mean()
median_val = df_clean['Time between scans (mths)'].median()

plt.axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')
plt.axvline(median_val, color='green', linestyle='-.', label=f'Median: {median_val:.2f}')

plt.xlabel('Time (months)')
plt.ylabel('Frequency')
plt.legend()
plt.grid()
plt.show()

In [ ]:
def make_table(df, stat='mdn', grp='grp', age='age', sex='sex', scan_dif='Time between scans (mths)', edu='education', eth='ethnicity', hand='handedness'):
    """
    Create a summary statistics table for the given DataFrame.
    """
    table = pd.DataFrame(columns=df[grp].unique())

    grp_values = list(df[grp].unique()) + ['ALL PX']
    for grp_value in grp_values:
        if grp_value == 'ALL PX':
            sub_df = df[df[grp] != 'CTRL'].copy()
        else:
            sub_df = df[df[grp] == grp_value].copy()

        # counts
        n_individuals = sub_df['MICS_ID'].nunique()
        table.loc['n', grp_value] = n_individuals

        # Ensure numeric for stats
        sub_df[age] = pd.to_numeric(sub_df[age], errors='coerce')
        sub_df[scan_dif] = pd.to_numeric(sub_df[scan_dif], errors='coerce')

        # Numerical: age (per study)
        for study in sub_df['study'].unique():
            study_mask = sub_df['study'] == study
            if stat == 'mdn':
                age_mdn = sub_df.loc[study_mask, age].median()
                age_iqr = sub_df.loc[study_mask, age].quantile(0.75) - sub_df.loc[study_mask, age].quantile(0.25)
                row_label = f'age_{study} (mdn (IQR))'
                table.loc[row_label, grp_value] = f"{age_mdn:.1f} ({age_iqr:.1f})"
            elif stat == 'mean':
                age_mean = sub_df.loc[study_mask, age].mean()
                age_std = sub_df.loc[study_mask, age].std()
                row_label = f'age_{study} (mean (std))'
                table.loc[row_label, grp_value] = f"{age_mean:.1f} ({age_std:.0f})"

        # remove repeated rows based on MICS_ID
        sub_df = sub_df.drop_duplicates(subset='MICS_ID')

        # Numerical: Time between scans (mths)
        if stat == 'mdn':
            tbs_mdn = sub_df[scan_dif].median()
            tbs_iqr = sub_df[scan_dif].quantile(0.75) - sub_df[scan_dif].quantile(0.25)
            row_label = 'Time between scans (mths) (mdn (IQR))'
            table.loc[row_label, grp_value] = f"{tbs_mdn:.1f} ({tbs_iqr:.1f})"
        elif stat == 'mean':
            tbs_mean = sub_df[scan_dif].mean()
            tbs_std = sub_df[scan_dif].std()
            row_label = 'Time between scans (mths) (mean (std))'
            table.loc[row_label, grp_value] = f"{tbs_mean:.1f} ({tbs_std:.1f})"

        # Cat: sex
        sub_df.loc[:, sex] = sub_df[sex].str.lower().map({'f': 'female', 'm': 'male'})
        n_females = (sub_df[sex] == 'female').sum()
        n_males = (sub_df[sex] == 'male').sum()
        n_total = sub_df[sex].notna().sum()
        pct_females = (n_females / n_total * 100) if n_total > 0 else 0
        row_label = 'sex (F:M (% F))'
        row_label_count = 'sex (F:M (%F))'
        table.loc[row_label_count, grp_value] = f"{n_females}:{n_males} ({pct_females:.0f}%)"

        # Cat: education
        edu_counts = sub_df[edu].value_counts(dropna=False)
        edu_str = ', '.join([f"{k}:{v}" for k, v in edu_counts.items()])
        row_label = 'education (count)'
        table.loc[row_label, grp_value] = edu_str

        # Cat: Handedness
        hand_counts = sub_df[hand].value_counts(dropna=False)
        n_left = hand_counts.get('L', 0)
        n_right = hand_counts.get('R', 0)
        hand_str = f"{n_left}:{n_right}"
        row_label = 'handedness (L:R)'
        table.loc[row_label, grp_value] = hand_str

        # Cat: lateralization of epileptogenic focus
        if grp_value != 'CTRL':
            lat_counts = sub_df['Lateralization of epileptogenic focus'].value_counts(dropna=False)
            n_tot = lat_counts.sum()
            # Report as L:R:unknown
            n_left = lat_counts.get('L', 0)
            n_left += lat_counts.get('L ', 0)  # Handle trailing space
            n_right = lat_counts.get('R', 0)
            n_right += lat_counts.get('R ', 0)  # Handle trailing space
            n_right += lat_counts.get('right', 0)  # Handle 'right' case
            n_left += lat_counts.get('left', 0)  # Handle 'left' case
            # Count L>R or R>L and add to L or R accordingly
            n_unclear = lat_counts.get('unclear', 0)
            n_unclear += lat_counts.get('unclear ', 0)
            
            print(f"Tot: {n_tot}, sum: {n_left + n_right + n_unclear}")
            
            if n_left + n_right + n_unclear != n_tot:
                print(f"\t {n_left + n_right + n_unclear} != {n_tot} for {grp_value}")
                n_bl = lat_counts.get('BL', 0)

                if n_bl > 0:
                    print("BL found in Lateralization of epileptogenic focus")
                    print(f"R, L counts: {n_right}, {n_left}")
                    # Subset rows with 'BL' in 'Lateralization of epileptogenic focus'
                    bl_rows = sub_df[sub_df['Lateralization of epileptogenic focus'] == 'BL']
                    # Search 'Dx at EMU discharge ' for 'L>' or 'R>' and add to counts
                    n_bl_l = bl_rows['Dx at EMU discharge '].astype(str).str.contains('L>').sum()
                    n_bl_r = bl_rows['Dx at EMU discharge '].astype(str).str.contains('R>').sum()
                    n_left += n_bl_l
                    n_right += n_bl_r
                    # Count all other BL cases that are not clearly L> or R>
                    n_bl_u = n_bl - (n_bl_l + n_bl_r)
                    print(f"R, L counts: {n_right}, {n_left}")
            else:
                n_bl_u = 0

            lat_str = f"{n_left}:{n_right}:{n_bl_u}:{n_unclear}"
        
        else:
            lat_str = ''
        
        row_label = 'Lateralization (L:R:BL (unclear dominance):unclear)'
        table.loc[row_label, grp_value] = lat_str

    return table


In [ ]:
t1 = make_table(df_clean, stat='mean', grp='grp')

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
t1
